In [1]:
import oracledb

connection = oracledb.connect(
    user = 'BD150224213',
    password = 'Hhqnm9',
    dsn = '172.16.12.14/xe',
)


In [2]:
cursor = connection.cursor()


In [44]:
cursor = connection.cursor()
cursor.execute('DROP TABLE produto')

cursor.execute("""
                CREATE TABLE produto(
                nome  VARCHAR2(255) NOT NULL ,
                descricao VARCHAR2(255),
                codigo VARCHAR2(30) NOT NULL PRIMARY KEY,
                custo INTEGER NOT NULL,
                custoFixo INTEGER NOT NULL,
                comissao INTEGER NOT NULL,
                imposto INTEGER NOT NULL,
                rentabilidade NUMBER NOT NULL 
                )"""
)

In [45]:
cursor = connection.cursor()

cursor.execute(""" 
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caneta', 'Caneta Profissional', '1', 36, 15, 5, 12, 20)
""")
cursor.execute(""" 
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Lapis', 'Preto B2', '2', 1, 1, 1, 1, 1)
""")

cursor.execute("""
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caderno', 'Palmeiras', '3', 10, 10, 10, 10, 50)
                """)



cursor.execute("""
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caderno', 'São Paulo', '4', 10, 10, 10, 10, 0)
                """)

cursor.execute("""
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caderno', 'Corinthians', '5', 10, 10, 10, 10, -20)
                """)

cursor.execute("""
                INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                VALUES ('Caderno', 'Ponte Preta', '6', 10, 30, 20, 20, 29.99)
                """)


connection.commit()



In [60]:
campos = ["nome", "descricao", "codigo", "custo", 'custoFixo', 'comissao', 'imposto', 'rentabilidade']
cod = ['1']
def seleciona_produto(a,b):
    cursor = connection.cursor()
    cursor.execute('SELECT ' + campos[selecionaProduto] + ' FROM PRODUTO WHERE(codigo =' +str(selecionaProduto)+')')
    resultado = cursor.fetchall()
    for row in resultado:
        print(row[0])
    cursor.close()
def exibir_menu():
    print(36 * "=")
    print("Menu:")
    print(36 * "=")
    print("1. Adicionar novo produto")
    print("2. Selecionar um produto")
    print("3. Sair")
    print(36 * "-")
def adicionar_novoProduto():
    print(36 * "=")
    print("\t Sistema de Cadastro")
    print(36 * "=")

    codProduto = input("Digite o código do produto: ")
    nomeProduto = input("Digite o nome do produto: ")
    descProduto = input("Adicione uma descrição ao produto: ")

    custoProduto = float(input("Qual o custo do Produto: "))
    custoFixoPct = float(input("Qual os custo fixos/administrativos do comércio [%]: "))
    comissaoVendaPct = float(input("Qual a comissão de venda do produto,em porcentagem [%]: "))
    impostoVendaPct = float(input("Qual a aliquota de imposto desejada [%]: "))
    margemLucroPct = float(input("Qual a margem de lucro desejada [%]: "))

    cursor = connection.cursor()

    cursor.execute(f""" 
                    INSERT INTO produto (nome, descricao, codigo, custo, custoFixo, comissao, imposto, rentabilidade) 
                    VALUES ('{nomeProduto}', '{descProduto}', '{codProduto}', {custoProduto}, {custoFixoPct}, {comissaoVendaPct}, {impostoVendaPct}, {margemLucroPct})
                """)
    connection.commit()
    
    print('Cadastro concluído com sucesso!')


def selecionar_produto():
    # Lógica para exibir as tarefas existentes
    selecionaProduto = input('Digite o código do produto: ')
    # Preço de venda produto
    cursor = connection.cursor()
    nomeBD = seleciona_produto(campos[0], selecionaProduto)
    custoProdutoBD = seleciona_produto(campos[3], selecionaProduto)
    comissaoVendaPctBD = seleciona_produto(campos[5], selecionaProduto)
    impostoVendaPctBD = seleciona_produto(campos[6], selecionaProduto)
    margemLucroPctBD = seleciona_produto(campos[7], selecionaProduto)
    

def main():
    while True:
        exibir_menu()
        
        opcao = input("Escolha uma opção: ")
        
        if opcao == "1":
            adicionar_novoProduto()
        elif opcao == "2":
            selecionar_produto()
        elif opcao == "3":
            break
        else:
            print("Opção inválida. Tente novamente.")

if __name__ == "__main__":
    main()

Menu:
1. Adicionar novo produto
2. Selecionar um produto
3. Sair
------------------------------------
Caneta Profissional
Caneta Profissional
Caneta Profissional
Caneta Profissional
Caneta Profissional
Menu:
1. Adicionar novo produto
2. Selecionar um produto
3. Sair
------------------------------------
Caneta Profissional
Caneta Profissional
Caneta Profissional
Caneta Profissional
Caneta Profissional
Menu:
1. Adicionar novo produto
2. Selecionar um produto
3. Sair
------------------------------------


In [50]:
def seleciona_produto(a,b):
    cursor = connection.cursor()
    cursor.execute('SELECT ' + campos[selecionaProduto] + ' FROM PRODUTO WHERE(codigo =' +str(selecionaProduto)+')')
    resultado = cursor.fetchall()
    for row in resultado:
        print(row[0])
    cursor.close()

In [32]:
cursor = connection.cursor()
cursor.execute('SELECT ' + campos[2] + ' FROM PRODUTO WHERE(codigo =' +cod[0]+')')
resultado = cursor.fetchall()
for row in resultado:
    print(row[0])
cursor.close()

1


In [53]:
impostoVendaPctBD = seleciona_produto(campos[6], selecionaProduto)
print(impostoVendaPctBD)

Caneta Profissional
None


In [58]:
a = seleciona_produto(campos[0],cod[0])
print(a)

('Caneta Profissional',)
None


In [24]:
campos[0]

'nome'

In [11]:
selecionaProduto = int(input('Digite o código do produto: '))
cursor = connection.cursor()
cursor.execute(f'SELECT nome FROM PRODUTO WHERE(codigo = {selecionaProduto})')
resultado = cursor.fetchall()
for row in resultado:
    print(row[0])
cursor.close()


IndexError: tuple index out of range